### Import modules

In [1]:
import pandas as pd 
import numpy as np

### Import data

In [2]:
path = "data/"
df = pd.read_csv(path + "train.csv")

### Create dummies for days between and days after anti 

In [3]:
df['d_days_between'] = df['days_between'].apply(lambda x: 0 if np.isnan(x) else 1)
df['d_days_after_anti'] = df['days_after_anti'].apply(lambda x: 0 if np.isnan(x) else 1)
df.fillna({'days_between':0,'days_after_anti':0 },inplace=True)

In [4]:
for idx, row in df.iterrows():
    try:
        num_rooms = len(row['room_list'].split(","))
    except:
        if np.isnan(row['room_list']):
            num_rooms = 0
        else:
            num_rooms = 1
    df.loc[idx, 'num_rooms'] = num_rooms
    
df.drop('room_list', axis=1, inplace=True)
df['num_rooms_b'] = df['num_rooms'].apply(lambda x: x if x <= 2 else 3)

### Create dummies for Gender

In [5]:
df_gender_dum = pd.get_dummies(df.Gender , prefix='gender_')
df = pd.concat([df, df_gender_dum], axis=1)

### Create dummies for Past positive result

In [6]:
df_Past_positive_dum = pd.get_dummies(df.Past_positive_result_from , prefix='Past_positive_')
df = pd.concat([df, df_Past_positive_dum], axis=1)
# df.drop(['Gender', 'Past_positive_result_from'], axis=1)

In [7]:
#compute days neutropenic without FN 
#dummie FN after being neutropenic
#all string dates to datetime (so we can compute)
def dates(df,days_neutropenic_wo_fn,dummie_days_neutropenic_wo_fn):
    dates_columns = ["start_neutropenico","start_FN","birth_year"]
    for c in dates_columns:
        df[c] = pd.to_datetime(df[c],format='%Y-%m-%d')
    
    df[days_neutropenic_wo_fn] = df["start_FN"] - df["start_neutropenico"]
    
    #conver to integer
    df[days_neutropenic_wo_fn] = df[days_neutropenic_wo_fn].dt.days
    #dummie if patient got FN after few days of having neutropenic status
    df[dummie_days_neutropenic_wo_fn] = np.where(df[days_neutropenic_wo_fn]>0, 1, 0)
    return df

In [8]:
dates(df,'days_neutropenic_wo_fn','dummie_days_neutropenic_wo_fn')


,MDR,ID,NHC,start_neutropenico,start_FN,days_between,days_in_hospital,hospital_stay_w_FN,prev_hospital_stay,birth_year,...,num_rooms_b,gender__female,gender__male,Past_positive__Blood culture,Past_positive__Both,Past_positive__Culture,Past_positive__NEGATIVE,Past_positive__NEITHER,days_neutropenic_wo_fn,dummie_days_neutropenic_wo_fn
0,0,374-1,404,2007-12-11,2008-01-01,0.0,28,1,3,1970-01-01 00:00:00.000001941,...,1.0,1,0,0,0,1,0,0,21,1
1,0,398-1,1897,2007-12-28,2008-01-01,0.0,8,1,6,1970-01-01 00:00:00.000001935,...,1.0,1,0,0,0,0,1,0,4,1
2,0,403-1,556,2008-01-01,2008-01-01,0.0,2,1,1,1970-01-01 00:00:00.000001980,...,1.0,0,1,0,0,0,1,0,0,0
3,0,407-1,454,2008-01-05,2008-01-05,0.0,1,1,9,1970-01-01 00:00:00.000001986,...,3.0,0,1,0,0,0,1,0,0,0
4,0,394-1,1615,2007-12-22,2008-01-06,0.0,17,1,5,1970-01-01 00:00:00.000001943,...,2.0,1,0,0,0,1,0,0,15,1
5,0,392-1,1723,2008-01-08,2008-01-08,0.0,20,1,1,1970-01-01 00:00:00.000001941,...,0.0,0,1,0,0,0,1,0,0,0
6,0,396-1,1521,2007-12-28,2008-01-12,0.0,22,1,3,1970-01-01 00:00:00.000001951,...,1.0,0,1,1,0,0,0,0,15,1
7,0,406-1,232,2008-01-11,2008-01-12,0.0,9,1,3,1970-01-01 00:00:00.000001950,...,1.0,1,0,0,0,0,1,0,1,1
8,0,415-1,38,2008-01-12,2008-01-12,0.0,2,1,4,1970-01-01 00:00:00.000001973,...,1.0,1,0,0,0,0,1,0,0,0
9,0,388-1,544,2007-12-25,2008-01-13,0.0,27,1,1,1970-01-01 00:00:00.000001947,...,1.0,0,1,0,0,1,0,0,19,1


In [9]:
df.to_csv('data/train_treated.csv')